## Libraries

In [1]:
%cd /content/drive/MyDrive/GenAI/OpenAI/Fine Tuning a GPT Model

/content/drive/MyDrive/GenAI/OpenAI/Fine Tuning a GPT Model


In [2]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [3]:
# Import the userdata module from Google Colab
from google.colab import userdata
# Retrieve the API key stored under 'genai_course' from Colab's userdata
api_key = userdata.get('genai_course')

In [11]:
import os
from os.path import isfile, join
import re
import pandas as pd
from openai import OpenAI
import random
import json

In [45]:
client = OpenAI(api_key = api_key)
MODEL = "gpt-4o"

## Train data preparation

In [54]:
# Get all the files in the 'emails data' directory
path = 'emails data'
files = [f for f in os.listdir(path) if isfile(join(path, f))]
files

['🚀 GPT-4o can do this.txt',
 '🚀 prompt engineering guide.txt',
 '💡 OpenAI is too Vanilla.txt',
 '🚀Free Course Alert Master Statistics with Excel.txt',
 '🤓 Statistics and your sphincter....txt',
 '🐍🔥 Hot Course Dropping Python for Data Analysis!.txt',
 '🌟 A Stellar Guide to Data Careers Which Path Will You Choose.txt',
 '🦌 baby reindeer and AI.txt',
 '👊 Stats with Python No Nonsense, Pure Skill!.txt',
 '💥 Temporal Fusion Transformer, explained.txt',
 '🚀Data Analyst Career Path, 5 Pandas Tricks and Data Science in 2024.txt',
 '🔓 Be Exceptional. Only 5_ Will Today—Will You.txt',
 '🤯 I am sure this would deceive you.txt',
 '🦍 Ape king mode on.txt',
 '🐛 Dune 2, spicy sausages and AI.txt',
 'The price of a Coca-Cola.txt']

In [55]:
# Open each email file and read its contents
emails = []
for em in files:
  data_dict = {}
  with open(f'emails data/{em}', 'r') as file:
      contents = file.read()

  title=contents.split('\n')[0]
  data_dict['content'] = f"Title: {title.replace('Title: ', '')}, Body: {contents.replace(title, '').strip()}"

  emails.append(data_dict)

In [56]:
emails[1]

{'content': "Title: 🚀 prompt engineering guide, Body: Reader,\n\nDo you, like Brother Elon, think AI poses a threat?\n\nAre you scared the robots are going to take over and dominate us like in the Terminator?\n\nDon't be.\n\nOr, better yet.\n\nBe more like Arnold and be back to terminate those mofos.\n\nImagine mastering a skill so powerful that it can multiply your results tenfold, transforming your business overnight.\n\nSounds like a fairytale, right?\n\nBut this isn't fantasy—it's the world of prompt engineering techniques.\n\nLet me break it down for you.\n\nEverything in AI starts with a prompt.\n\nWhether it's one-shot, few-shot, chaining, or the tree of thoughts, these techniques are the secret sauce that can turbocharge your AI's performance.\n\nOne-shot learning is like a master chef whipping up a gourmet dish from a single glance at the ingredients.\n\nWith just one example, you can train your model to understand complex tasks. Imagine the time you'll save!\n\nFew-shot learn

In [57]:
# Define the system prompt
system_prompt = """
You are an expert prompt engineer and content creator.
Analyze the email and draft a prompt up to 100 characters that would have led to that email.
write just the content of the prompt and nothing else.
"""

In [58]:
# Use the GPT model to extract the main subject of the email
prompts = []

for mail in emails:
  response = client.chat.completions.create(
    model=MODEL,
    messages=[
       {"role": "system", "content": system_prompt},
      {"role": "user", "content": mail['content']}])
  print(response.choices[0].message.content)
  prompts.append(response.choices[0].message.content)

Announce GPT-4o's ability to convert video to text and link to tutorial.
Create an email about mastering prompt engineering techniques for exponential growth.
Email about a startup event focused on AI, customization, and OpenAI insights.
Write an email promoting a free course on mastering statistics with Excel.
Write a captivating email promoting a statistics mastery course.
Launch a new Python for Data Analysis course; engage learners with humor and urgency.
Write an electrifying email unveiling a must-read blog for data career growth.
Draft a fun email promoting a data mining course using a TV show comparison.
Prompt: Write an engaging email about a new Python statistics course launch.
Draft an engaging email promoting a YouTube video on Temporal Fusion Transformers.
Promote a data analyst career path with courses and earning potential information.
Prompt: "Create an email encouraging productivity with exclusive content and motivational tone."
Prompt: Create a persuasive email warnin

In [59]:
# Combine emails and prompts into a single list
combined_data = list(zip(emails, prompts))

In [60]:
# Shuffle the combined data (emails and prompts retain the same index)
random.shuffle(combined_data)

In [61]:
# Split data into training and validation sets (80-20 split)
train_size = int(0.8 * len(combined_data))
train_data_combined = combined_data[:train_size]
validation_data_combined = combined_data[train_size:]

In [62]:
# Prepares a message structure for training, containing system, user, and assistant messages
def prepare_data_json(data, subject):
  system_msg = """
  You are a skilled social media manager specialized in Analytics and AI.
  Your main task is to write engaging, concise, and persuasive emails for subscribers"""

  return {
    "messages": [
        {"role": "system", "content": system_msg},
        {"role": "user", "content":  f"Write an email about: {subject}"},
        {"role": "assistant", "content": f"Email: {data['content']}"},
    ]
  }

In [63]:
# Prepare training examples
train_data = []
for email, prompt in train_data_combined:
    train_data.append(prepare_data_json(email, prompt))

In [64]:
# Prepare validation examples
validation_data = []
for email, prompt in validation_data_combined:
    validation_data.append(prepare_data_json(email, prompt))

In [65]:
# Write the training and validation data into .jsonl files
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [66]:
# write training data to jsonl
write_jsonl(train_data, 'train_data.jsonl')

# write validation data to jsonl
write_jsonl(validation_data, 'validation_data.jsonl')

In [67]:
# preview
#!head -n 5 train_data.jsonl
!head train_data.jsonl

{"messages": [{"role": "system", "content": "\n  You are a skilled social media manager specialized in Analytics and AI.\n  Your main task is to write engaging, concise, and persuasive emails for subscribers"}, {"role": "user", "content": "Write an email about: Write an electrifying email unveiling a must-read blog for data career growth."}, {"role": "assistant", "content": "Email: Title: \ud83c\udf1f A Stellar Guide to Data Careers Which Path Will You Choose, Body: Hey Reader\n\nSweet child, let me be brief\u2026\n\nBecause I\u2019m about to drop something seismic.\n\nToday, I\u2019m unveiling a gem on Zero To Mastery that\u2019s set to send shockwaves from Silicon Valley to the farthest reaches of cyberspace.\n\nThis isn't your average snooze-fest of data babble.\n\nNah.\n\nI am talking about the holy grail for Data Engineers, Analysts, and Scientists.\n\nA blog post so riveting, it\u2019ll give the tech gurus and data nerds a collective mindgasm.\n\nImagine, with just a click, unloc

## Upload file to openAI & create fine tuning jobs

In [68]:
# Uploads a file to the server with the specified purpose and returns the file ID
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id

In [69]:
training_file_id = upload_file('train_data.jsonl', "fine-tune")
validation_file_id = upload_file('validation_data.jsonl', "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-3DDgXmnSNVHNKuTxRRpS0euJ
Validation file ID: file-xqJm8t5ZGJc8Nh07zIjdZs0R


In [70]:
MODEL = "gpt-4o-2024-08-06"

# Create a fine-tuning job using the training file
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model=MODEL,
    suffix="email-writer-Data Heroes",
    validation_file = validation_file_id
)

job_id = response.id

In [71]:
response

FineTuningJob(id='ftjob-2cKp14XetvJ5x9NnnNERhshu', created_at=1729611577, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-3CcTXbA3QFCpik5qtruD3kTB', result_files=[], seed=1677922270, status='validating_files', trained_tokens=None, training_file='file-3DDgXmnSNVHNKuTxRRpS0euJ', validation_file='file-xqJm8t5ZGJc8Nh07zIjdZs0R', estimated_finish=None, integrations=[], user_provided_suffix='email-writer-Data Heroes')

In [72]:
tuned_model = client.fine_tuning.jobs.retrieve(job_id)
# Retrieve the fine-tuning job details using the job ID

fine_tuned_model_id = tuned_model.fine_tuned_model
# Extract the fine-tuned model ID from the response

if fine_tuned_model_id is None:
    # Raise an error if the fine-tuning job hasn't been completed or no model ID is available
    raise RuntimeError(
        "Fine-tuned model ID not found. Your job has likely not been completed yet."
    )

print("Fine-tuned model ID:", fine_tuned_model_id)
# Print the ID of the fine-tuned model

RuntimeError: Fine-tuned model ID not found. Your job has likely not been completed yet.

In [ ]:
# Retrieve the fine-tuning job details using the job ID
job_details = client.fine_tuning.jobs.retrieve(job_id)

# Print the training and validation losses for each epoch
if "events" in job_details:
    for event in job_details["events"]:
        if event["level"] == "info":
            if "training_loss" in event:
                print(f"Epoch {event['epoch']} - Training Loss: {event['training_loss']}")
            if "validation_loss" in event:
                print(f"Epoch {event['epoch']} - Validation Loss: {event['validation_loss']}")


## Trial

In [73]:
# Define the system and user message
system_prompt = """You are a skilled social media manager specialized in Analytics and AI.
                  Your main task is to write engaging, concise, and persuasive emails for subscribers"""

topic = "New GenAI course section: Fine Tuning an GPT model"

user_prompt = f"Write an email about: {topic}"

In [74]:
test_messages = [{"role": "system",
                  "content": system_prompt},
                   {"role": "user",
                    "content": user_prompt}]


In [75]:
response = client.chat.completions.create(
    model=fine_tuned_model_id,
    messages=test_messages,
    temperature=1,
    #max_tokens=700
)
print(response.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': 'you must provide a model parameter', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [41]:
topic = "New AI Agent framework: OpenAI swarm"

test_messages2 = [{"role": "system",
                  "content": system_prompt},
                   {"role": "user",
                    "content": user_prompt}]


In [42]:
response = client.chat.completions.create(
    model=fine_tuned_model_id,
    messages=test_messages2,
    temperature=1
)
print(response.choices[0].message.content)

Email: Fine-Tuning GPT and Better CTR, Title:️ 🔧 Fine-Tuning GPT, Reader Attention & Top 1%, Body: Reader,

You know I’m all about getting hands-on with tech.

Diving deep, getting my hands dirty, and understanding how stuff really works.

Remember that AI Hype course I told you about?

Well, I’ve been stuck in, expanding it with sections that really add value.

And there’s one area I’m buzzing to tell you about: Fine-Tuning GPT.

This ain’t just theory or fluff—this is actionable, game-changing knowledge.

But why, you ask, should Fine-Tuning GPT matter to you?

Let’s break it down.


1. Fine-Tuning GPT: Making Cutting-Edge More Personal

Fine-Tuning, Explained: Think of it as customizing a suit. You start with something standard and tweak it until it fits like a glove. With GPT models, this means adjusting them so they perform perfectly for specific tasks or industries.

Save Money, Get Better Results: Fine-tuning can be way more cost-effective than training a model from scratch, and

# Validation

In [43]:
# Use validation data to assess model performance
for mail in validation_emails:
    test_prompt = mail['content']
    test_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Write an email about: {test_prompt}"}
    ]
    response = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=test_messages,
        temperature=1
    )
    print("\nValidation Topic:", test_prompt)
    print("Generated Email:", response.choices[0].message.content)


Validation Topic: Title: 🚀 GPT-4o can do this, Body: You won't believe this.

GPT-4o can read videos!

Yeah, you heard that right.

God damn video-to-text.

OpenAI's latest API can take any video and convert it into text, turning hours of manual work into minutes.

In my latest tutorial, I’m breaking down how to:

Integrate OpenAI API with Python
Seamlessly convert video content to text
Use Whisper to transcribe audio and supercharge GPT-4o
Ready to level up?

Watch the tutorial now: https://youtu.be/JQ_Er9bMLPk​


Catch you on the inside!

Diogo

PS: Don't forget to watch.
Generated Email: Email: Title: 🌟 GPT-4o's Jaw-Dropping Skill, Body: Can it really do this?, GPT-4o can read videos!

Yeah, you heard that right.

God damn video-to-text.

OpenAI's latest API can take any video and convert it into text, turning hours of manual work into minutes.

In my latest tutorial, I’m breaking down how to:

Integrate OpenAI API with Python
Seamlessly convert video content to text
Use Whisper to